In [ ]:
## pip install phonenumbers # if you do not have it

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [289]:
# df1 = pd.read_csv('/Users/noeoberle/Boom chicago/FareHarbour CSV Files/Vsae--2025-01-01--2025-12-31.csv', header = 1)
# df2 = pd.read_csv('/Users/noeoberle/Boom chicago/FareHarbour CSV Files/Vsae--2024-01-01--2025-01-01.csv', header = 1)
# df3 = pd.read_csv('/Users/noeoberle/Boom chicago/FareHarbour CSV Files/Vsae--2023-01-01--2024-01-01.csv', header = 1)
# df4 = pd.read_csv('/Users/noeoberle/Boom chicago/FareHarbour CSV Files/Vsae--2022-01-01--2023-01-01.csv', header = 1)

df1 = pd.read_csv('Vsae--2022-01-01--2023-01-01.csv', header = 1)
df3 = pd.read_csv('Vsae--2024-01-01--2025-01-01.csv',header = 1)
df4 = pd.read_csv('Vsae--2025-01-01--2025-12-31.csv', header = 1)
df2 = pd.read_csv('Vsae--2023-01-01--2024-01-01.csv', header = 1)

In [291]:
#df1

In [293]:
#df2

In [295]:
merged_df = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)


In [297]:
#merged_df

In [329]:
clean_df = merged_df.drop_duplicates()

In [301]:
#clean_df

In [331]:
clean_df = clean_df.drop(columns = ['Contact', 'Booking created by', 'Last Booked By', 'Affiliate', 'Total Tax', 'Total', 'Total Paid', 'Processing Fees', 'Total Paid after Processing Fees'])

In [333]:
import phonenumbers

def extract_country_code(number):
    if pd.isna(number): # kept all the NA's to make sure there is no shift
        return np.nan
    try:
        parsed = phonenumbers.parse(number, 'NL')  # NL default
        return f'+{parsed.country_code}'
    except phonenumbers.NumberParseException:
        return '+31'  # NL default

clean_df['Phone'] = clean_df['Phone'].apply(extract_country_code)

In [334]:
# Ignore how bad this looks <3, it works
showlist = ['Improv Spectacular | Comedy Show',
 'Last Minute Stand-Ins',
 'Shot of Improv',
 'Sunday Night Live',
 'The Famous Boom Chicago Improv Comedy Show',
 'Tragedy + Time = Comedy',
 'WTF Improv',
 'Laugh Out Proud',
 'The Comedy Embassy',
 'The Good, The Bad, and The Algorithm']

clean_df['is_Improvspec'] = (clean_df["Item"] == showlist[0]).astype(int)
clean_df['is_lastminute'] = (clean_df["Item"] == showlist[1]).astype(int)
clean_df['is_Shots'] = (clean_df["Item"] == showlist[2]).astype(int)
clean_df['is_Sundaylive'] = (clean_df["Item"] == showlist[3]).astype(int)
clean_df['is_Famousimprov'] = (clean_df["Item"] == showlist[4]).astype(int)
clean_df['is_Tragedy'] = (clean_df["Item"] == showlist[5]).astype(int)
clean_df['is_Wtf'] = (clean_df["Item"] == showlist[6]).astype(int)
clean_df["is_Laughproud"] = (clean_df["Item"] == showlist[7]).astype(int)
clean_df['is_Comedyembassy'] = (clean_df["Item"] == showlist[8]).astype(int)
#is_Goodbadalgo = (clean_df["Item"] == showlist[9]).astype(int) # Let this be the base case, leave this column out

In [335]:
# removes non-show items
for i, show in enumerate(clean_df['Item']):
    if show not in showlist:
        clean_df = clean_df.drop(index = i)

In [336]:
clean_df['Availability'] = clean_df['Availability'].str.replace(r'^.*@ ', '', regex=True)

In [545]:
clean_df.rename(columns={'Availability': 'Show Time'}, inplace=True)
# clean_df

'7:30pm – 9:30pm'

In [341]:
clean_df['Availability date'] = clean_df['Availability date'].str.replace('/', '-', regex=False)

# Convert to datetime safely
clean_df['Availability date'] = pd.to_datetime(
    clean_df['Availability date'], 
    dayfirst=True, 
    errors='coerce'
)

#clean_df

In [454]:
# df_weather = pd.read_csv('/Users/noeoberle/Boom chicago/Weather/export.csv')
df_weather = pd.read_csv('export.csv')
clean_df_weather = df_weather.drop(columns = ['tmin', 'tmax', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun'])
clean_df_weather['date'] = pd.to_datetime(clean_df_weather['date'])

clean_df_weather['date'] = clean_df_weather['date'].dt.date
#clean_df_weather

In [456]:
clean_df_weather['date'] = pd.to_datetime(df_weather['date'])
clean_df['Availability date'] = pd.to_datetime(clean_df['Availability date'])

# Merge on the date
df_combined = pd.merge(
    clean_df, 
    clean_df_weather, 
    left_on='Availability date', 
    right_on='date', 
    how='left'
)


#df_combined

In [458]:
df_combined['date'] = pd.to_datetime(df_combined['date'])


df_combined['weekday'] = df_combined['date'].dt.day_name()

# df_combined

In [460]:
df_combined['weekend_flag'] = df_combined['weekday'].isin(['Friday', 'Saturday', 'Sunday']).astype(int)
# df_combined

In [462]:
df_combined['# of Nationals'] = (df_combined['Phone'] == '+31')*df_combined['# of Pax']

In [464]:
df_combined['Subtotal'] = (
    df_combined['Subtotal']
    .str.replace('€', '', regex=False)     
    .str.replace(',', '', regex=False)     
    .astype(float)                         
)

grouped_df = df_combined.groupby(['Item', 'date']).agg({
    'Show Time': 'first',
    '# of Pax': 'sum',
    'Subtotal': 'sum',
    'Booking ID': 'count',
    'tavg': 'mean',
    'prcp': 'mean',
    'weekday': 'first',
    'weekend_flag': 'first',
    '# of Nationals': 'sum',
    'is_Improvspec': 'first', # dummy variables for showtype
    'is_lastminute': 'first',
    'is_Shots': 'first',
    'is_Sundaylive': 'first',
    'is_Famousimprov': 'first',
    'is_Wtf': 'first',
    'is_Laughproud': 'first',
    'is_Comedyembassy': 'first'
}).reset_index()


grouped_df.rename(columns={
    '# of Pax': 'total_pax',
    'Subtotal': 'total_revenue',
    'Booking ID': 'num_bookings',
    'tavg': 'avg_temp',
    'prcp': 'avg_prcp'
}, inplace=True)

grouped_df['Perc_Nationals'] = 100*grouped_df['# of Nationals']/grouped_df['total_pax']
grouped_df = grouped_df.drop(columns='# of Nationals')
# grouped_df

In [466]:
grouped_df

,Item,date,Show Time,total_pax,total_revenue,num_bookings,avg_temp,avg_prcp,weekday,weekend_flag,is_Improvspec,is_lastminute,is_Shots,is_Sundaylive,is_Famousimprov,is_Wtf,is_Laughproud,is_Comedyembassy,Perc_Nationals
0,Improv Spectacular | Comedy Show,2022-01-29,7:30pm – 9:30pm,137,2448.11,86,9.3,0.0,Saturday,1,1,0,0,0,0,0,0,0,75.912409
1,Improv Spectacular | Comedy Show,2022-02-05,7:30pm – 9:30pm,85,1688.80,48,6.5,4.3,Saturday,1,1,0,0,0,0,0,0,0,80.000000
2,Improv Spectacular | Comedy Show,2022-02-10,7:30pm – 9:30pm,6,114.68,3,6.8,NaN,Thursday,0,1,0,0,0,0,0,0,0,100.000000
3,Improv Spectacular | Comedy Show,2022-02-11,7:30pm – 9:30pm,29,908.28,18,4.7,1.6,Friday,1,1,0,0,0,0,0,0,0,79.310345
4,Improv Spectacular | Comedy Show,2022-02-12,7:30pm – 9:30pm,83,2556.86,51,2.6,0.0,Saturday,1,1,0,0,0,0,0,0,0,72.289157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,WTF Improv,2025-02-26,8pm – 10pm,93,1333.32,37,6.8,2.6,Wednesday,0,0,0,0,0,0,1,0,0,76.344086
1136,WTF Improv,2025-03-19,8pm – 10pm,70,1014.62,26,7.9,0.0,Wednesday,0,0,0,0,0,0,1,0,0,60.000000
1137,WTF Improv,2025-03-26,8pm – 10pm,99,517.34,22,8.4,0.0,Wednesday,0,0,0,0,0,0,1,0,0,17.171717
1138,WTF Improv,2025-04-02,8pm – 10pm,67,716.74,27,11.6,0.0,Wednesday,0,0,0,0,0,0,1,0,0,29.850746


In [565]:
# Shot of Improv has weird Show Times!!!
vec = []
for i in range(len(grouped_df['Show Time'])-1):
    if grouped_df['Show Time'][i] != grouped_df['Show Time'][i+1]:
        vec.append(i)
vec.append(1139)

# for item in vec:
#     print(item, grouped_df['Item'][item], grouped_df['date'][item], grouped_df['Show Time'][item], '\n')

grouped_df.iloc[475]

Item                     Shot of Improv
date                2024-11-02 00:00:00
Show Time             10:15pm – 3/11/24
total_pax                            57
total_revenue                    773.72
num_bookings                         29
avg_temp                           10.4
avg_prcp                            0.2
weekday                        Saturday
weekend_flag                          1
is_Improvspec                         0
is_lastminute                         0
is_Shots                              1
is_Sundaylive                         0
is_Famousimprov                       0
is_Wtf                                0
is_Laughproud                         0
is_Comedyembassy                      0
Perc_Nationals                68.421053
Name: 475, dtype: object